In [1]:
import sys
sys.path.append('../02_dataset_generation')

In [2]:
import json

In [3]:
from schema_to_sqlalchemy import tables_to_sqlalchemy_file

In [4]:
with open('../db_schemas.json', 'r') as f:
    schemas = json.load(f)

In [5]:
tables = schemas[1].get('tables')

In [6]:
await tables_to_sqlalchemy_file(tables, 'output_sqlalchemy_schema.py')

In [7]:
from output_sqlalchemy_schema import create

In [8]:
engine = create()

In [9]:
import pandas as pd

In [11]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", engine)
tables

,name
0,MenuItems
1,Orders
2,OrderItems
3,Staff
4,Customers


In [12]:
pd.read_sql(f"SELECT * FROM {tables.name.sample(1).iloc[0]}", engine)

,menu_item_id,item_name,description,category,price,is_available


In [13]:
from create_data import populate_data

In [14]:
await populate_data('output_sqlalchemy_schema.py', 'populate_data.py')

In [15]:
from populate_data import populate

In [16]:
populate(engine)

In [ ]:
from IPython.display import display

In [17]:
for table in tables.name:
    display(pd.read_sql(f"SELECT * FROM {table}", engine))


,menu_item_id,item_name,description,category,price,is_available
0,1,Margherita Pizza,"Classic tomato, mozzarella, and basil pizza.",Pizza,12.99,1
1,2,Pepperoni Pizza,Pizza with pepperoni slices.,Pizza,14.99,1
2,3,Vegetarian Pizza,Pizza with assorted vegetables.,Pizza,13.99,1
3,4,Chicken Caesar Salad,"Romaine lettuce, grilled chicken, parmesan, cr...",Salads,9.99,1
4,5,Greek Salad,"Tomatoes, cucumbers, onions, olives, and feta ...",Salads,8.99,1
5,6,Cheeseburger,"Beef patty, cheese, lettuce, tomato, and onion...",Burgers,10.99,1
6,7,Bacon Cheeseburger,"Beef patty, cheese, bacon, lettuce, tomato, an...",Burgers,12.99,1
7,8,Fries,Classic French fries.,Sides,4.99,1
8,9,Onion Rings,Crispy fried onion rings.,Sides,5.99,1
9,10,Coca-Cola,Classic Coca-Cola.,Drinks,2.99,1


,order_id,customer_id,staff_id,order_date,total_amount,payment_method,order_status
0,1,1,4,2025-05-06 14:49:13.359165,32.96,Online Payment,Shipped
1,2,3,3,2025-05-02 14:49:13.361674,25.98,Credit Card,Shipped
2,3,3,2,2025-05-17 14:49:13.362007,35.97,Online Payment,Shipped
3,4,2,3,2025-05-16 14:49:13.362159,22.97,Credit Card,Delivered
4,5,4,1,2025-04-25 14:49:13.362295,18.95,Online Payment,Received
5,6,2,3,2025-05-03 14:49:13.362534,9.98,Online Payment,Shipped
6,7,4,4,2025-05-11 14:49:13.362546,8.99,Credit Card,Shipped
7,8,4,3,2025-05-18 14:49:13.362556,63.42,Cash,Shipped
8,9,1,1,2025-05-14 14:49:13.362565,114.39,Credit Card,Received
9,10,5,4,2025-05-22 14:49:13.362574,70.92,Cash,Received


,order_item_id,order_id,menu_item_id,quantity,item_price
0,1,1,7,2,12.99
1,2,1,12,2,3.49
2,3,2,7,2,12.99
3,4,3,1,2,12.99
4,5,3,4,1,9.99
5,6,4,8,2,4.99
6,7,4,1,1,12.99
7,8,5,14,2,4.99
8,9,5,11,3,2.99
9,10,6,14,2,4.99


,staff_id,first_name,last_name,position,hire_date,salary
0,1,Alice,Smith,Manager,2020-01-15,60000.0
1,2,Bob,Johnson,Chef,2021-05-20,50000.0
2,3,Charlie,Brown,Waiter,2022-09-10,35000.0
3,4,David,Lee,Waiter,2023-01-05,36000.0


,customer_id,first_name,last_name,phone_number,email
0,1,Eve,Williams,555-123-4567,eve.williams@example.com
1,2,Frank,Davis,555-987-6543,frank.davis@example.com
2,3,Grace,Miller,555-246-8013,grace.miller@example.com
3,4,Henry,Wilson,555-135-7924,henry.wilson@example.com
4,5,Ivy,Moore,555-864-2097,ivy.moore@example.com


In [45]:
from generate_questions import generate_questions

In [46]:
questions = await generate_questions('output_sqlalchemy_schema.py')

In [66]:
item = questions.questions[12]
print(item.question)
print(item.sql)

Find the order IDs that contain more than 2 different menu items.
SELECT order_id FROM OrderItems GROUP BY order_id HAVING COUNT(DISTINCT menu_item_id) > 2;


In [67]:
pd.read_sql(item.sql, engine)

,order_id
0,8
1,9
2,10
